# In this notebook, we play with measured weather data and measured tree temperature, and try to fit a simple linear regression model. We will use 70% of data for training the model, and 30% to test the model.

In [1]:
import numpy as np
import pandas
from scipy import sparse
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
%matplotlib inline

import sys
print("Python 3 version is", sys.version)
import matplotlib
print("Matplotlib version is", matplotlib.__version__)
print("Numpy version is", np.__version__)
import scipy

print("Scipy version is", scipy.__version__)

Python 3 version is 3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Matplotlib version is 3.1.1
Numpy version is 1.17.2
Scipy version is 1.3.1


## Import data and linear interpret to be of same length. 

In [2]:
# define parameter
n_t = 4000 
n_x = 2000
radius = 0.135 # unit m

#### Read measured temp data-- only need bark and a mid point for validation

colnames = ['datetime', 's45_1', 'e9_1', 'n135_1','e45_2', 'n9_2', 'w135_2', 'n45_3', 'w9_3','s135_3', 'w_ext_35']
dataTemp = pandas.read_excel('Tree_Temp_Values_OCT21_to_OCT28_2022.xlsx', names=colnames)

    ##########
coreTemp = dataTemp.s135_3[2:416].tolist() # Oct 21 -- core temp is west, at 13.5cm, at 2m high
coreTempnp = np.asarray([float(t) for t in coreTemp])
coreTempnp = coreTempnp + 273.15
core_temp = np.interp(np.linspace(0,24,n_t), np.linspace(0,24,coreTempnp.size),coreTempnp)

    ##########
midTemp1 = dataTemp.w9_3[2:416].tolist() # Oct 21 -- West, at 9cm, at 3m high
midTemp1np = np.asarray([float(t) for t in midTemp1])
midTemp1np = midTemp1np + 273.15
mid_temp1 = np.interp(np.linspace(0,24,n_t), np.linspace(0,24,midTemp1np.size),midTemp1np)
    ##########
    
midTemp2 = dataTemp.n45_3[2:416].tolist() # Oct 21 -- North, at 4.5cm, at 3m high
midTemp2np = np.asarray([float(t) for t in midTemp2])
midTemp2np = midTemp2np + 273.15
mid_temp2 = np.interp(np.linspace(0,24,n_t), np.linspace(0,24,midTemp2np.size),midTemp2np)

    ##########
    
barkTemp = dataTemp.w_ext_35[2:416].tolist() # Oct 21 -- bark temp is West, at bark, at 3.5m high
barkTempnp = np.asarray([float(v) for v in barkTemp])
barkTempnp = barkTempnp + 273.15
bark_temp = np.interp(np.linspace(0,24,n_t), np.linspace(0,24,barkTempnp.size),barkTempnp)
    
    ##########
    
initTemp = np.array([core_temp[0], mid_temp1[0], mid_temp2[0], bark_temp[0]])
init_temp = np.interp(np.linspace(0,radius,n_x), np.linspace(0,radius,initTemp.size),initTemp)
    ##########
print("testing data read")

testing data read


In [3]:
### Read weather data-- do not need when use measured bark data as outer boundary condition

colnames = ['datetime', 'wind_speed', 'wind_direction', 'humidity', 'temperature', 'air_pressure', 'solar_DNI']
dataWeather = pandas.read_excel('Weather_Station_OCT21_to_OCT28_2022.xlsx', names = colnames)
   
temp = dataWeather.temperature[2:145].tolist() # use data from Oct 21
tempnp = np.asarray([float(t) for t in temp])
tempnp = tempnp + 273.15

windspeed = dataWeather.wind_speed[2:145].tolist()
windspeednp = np.asarray([float(v) for v in windspeed])

solar = dataWeather.solar_DNI[2:145].tolist()
solarnp = np.asarray([float(v) for v in solar])

air_temp = np.interp(np.linspace(0,24,n_t), np.linspace(0,24,tempnp.size),tempnp)
wind_speed = np.interp(np.linspace(0,24,n_t), np.linspace(0,24,windspeednp.size),windspeednp)
solar_input = np.interp(np.linspace(0,24,n_t), np.linspace(0,24,solarnp.size),solarnp)

print("weather data read")

weather data read


## Regression problem: with the input core, bark temperatures, as well as the weather data, predict temperature at midTemp1 or midTemp2.

\begin{equation}
\rho c\frac{\partial T}{\partial t}=\frac{1}{r}\frac{\partial}{\partial r}\bigg(kr\frac{\partial T}{\partial r}\bigg)+\text{ source terms}
=\frac{\partial k}{\partial r}\frac{\partial T}{\partial r}+\frac{k}{r}\frac{\partial T}{\partial r}+k\frac{\partial^2T}{\partial r^2}+\text{ source terms}.\label{heat1d}
\end{equation}

SyntaxError: invalid syntax (<ipython-input-4-130bfe158235>, line 1)